In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from collections import defaultdict

In [2]:
#class Grid:
    #==================================#
    # Python Wrapper Class for 2D npArray
    # Stores values representing availibility
    # Expects tuple(x,y) format of values
    #==================================#
#    def __init__(self, size):
#        self.size = size
#        self.grid = np.empty((size, size), dtype= np.ulonglong)
    
#    def insert(self, Person_id, times):
#        self.grid[times] += 1*10**Person_id
#    def delete(self, Person_id, times):
#        self.grid[times] -= 1*10**Person_id
        
#    def insertMult(self, Person_id, times):
#        self.grid[tuple(zip(*times))] += 1*10**Person_id
        
#    def deleteMult(self, Person_id, times):
#        self.grid[tuple(zip(*times))] -= 1*10**Person_id
    
#    def isAvailable(self, Person_id, time):
#        if Person_id >= len(str(self.grid[time])):
#            return False
#        return str(self.grid[time])[::-1][Person_id] == '1'
    
#   def getTotal(self, time):
#       return sum(map(int, str(self.grid[time]) ))
    
#   def getNextAvailable(self, Person_id, time):
        
#       baseStr = str(self.grid[time])[::-1]
#       if Person_id >= len(baseStr):
#           return baseStr.rfind('1')
        
#       if baseStr[Person_id] == '1':
#           return Person_id
        
#       back = baseStr[Person_id:]
#       front = baseStr[:Person_id]
        #print(baseStr)
        #print("front: ", front, "distance: ", front[::-1].find("1"))
        #print("back: ", back, "distance: ", back.find("1"))
        
#       frontL = front[::-1].find("1")
#       backL = back.find("1")
        
#       if backL == -1 and frontL == -1:
#           return -1
#       elif backL < frontL or frontL == -1:
#           return Person_id+backL
#       elif frontL > backL or backL == -1:
#           return Person_id - frontL
        
#       return (Person_id-frontL, Person_id+backL)
    
#   def print(self):
#       print(self.grid)

In [3]:
class Grid:
    def __init__(self, size):
        self.size = size
        self.Grid = pd.DataFrame(None, columns= [(x//size, x%size) for x in range(size**2)], dtype = bool)
        
    def insert(self, idx, times: list):
            
        self.Grid.loc[idx] = [ x in set(times) for x in self.Grid.columns]
        
    def delete(self, idx):
        if idx in self.Grid.index:
            self.Grid.drop(index=idx, inplace= True)
        else:
            print("Not in index")
    
    def change(self, idx, times: list, value):
        self.Grid.loc[idx] = [ value if x in set(times) else y for x,y in zip(self.Grid.columns, self.Grid.loc[idx])]
    
    def isAvailable(self, idx, times: list):
        if idx not in self.Grid.index:
            print("Not in index")
            return False
        
        pos = self.Grid.index.get_loc(idx)
        return self.Grid.iloc[pos, times[0]*self.size + times[1]]
    
    def getTotal(self, times):
        return self.Grid[times].sum()
    def getTotalbyidx(self, idx):
        return self.Grid.loc[idx].sum()
    def getTotalSpots(self):
        return len(self.Grid.columns)
    def getAvailable(self, time):
        return self.Grid.loc[self.Grid[time] == True][time]
    
#    def getNextAvailable(self, time):
#        ser = self.getAvailable(time)
#        ser = ser.to_frame()
#        arr = []
#        for x in ser.index:
#            arr.append(self.getTotalbyidx(x))
            
#        ser["vals"] = arr
#        return ser.sort_values(by = "vals")

    def getNextAvailable(self, time):
        ser = self.getAvailable(time)
        return ser.sort_index(key = lambda x: [self.getTotalbyidx(y) for y in x])
    def print(self):
        print(self.Grid)
    
class Person:
    newid = itertools.count()
    def __init__(self):
        self.id = next(Person.newid)
        self.list = set()
        
    def getList(self):
        return self.list
    
    def addtoList(self, hh, mm):
        self.list.append((hh, mm))
    def removeFromList(self, hh, mm):
        if (hh, mm) in self.list:
            self.list.remove((hh,mm))
        else:
            print("not in list")
            
    def inList(self, hh, mm):
        return (hh, mm) in self.list
            

In [4]:
print (pd.options.display.max_columns)
print (pd.options.display.max_rows)

20
60


In [5]:
test = Grid(4)
#test.print()
test.insert(20, [(0,0),(0,3),(2,2)])

test.insert(0, [(0,3)])
test.insert(3, [(0,0)])
test.change(3, [(1,0)], True)

print("Test is Available: ", test.isAvailable(3, (1,1)), "\n" )
display(test.getAvailable((0,0)) )
print("\nTest Total by Row: ", test.getTotalbyidx(20))
print("Test Total by Column: ", test.getTotal((0,0)), "\n" )

display(test.getNextAvailable((0,0)) )
display(test.Grid)

Test is Available:  False 



20    True
3     True
Name: (0, 0), dtype: bool


Test Total by Row:  3
Test Total by Column:  2 



3     True
20    True
Name: (0, 0), dtype: bool

,"(0, 0)","(0, 1)","(0, 2)","(0, 3)","(1, 0)","(1, 1)","(1, 2)","(1, 3)","(2, 0)","(2, 1)","(2, 2)","(2, 3)","(3, 0)","(3, 1)","(3, 2)","(3, 3)"
20,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False
0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


In [6]:
test.delete(4)
test.getTotal((2,2))

Not in index


1

In [7]:
test.print()

    (0, 0)  (0, 1)  (0, 2)  (0, 3)  (1, 0)  (1, 1)  (1, 2)  (1, 3)  (2, 0)  \
20    True   False   False    True   False   False   False   False   False   
0    False   False   False    True   False   False   False   False   False   
3     True   False   False   False    True   False   False   False   False   

    (2, 1)  (2, 2)  (2, 3)  (3, 0)  (3, 1)  (3, 2)  (3, 3)  
20   False    True   False   False   False   False   False  
0    False   False   False   False   False   False   False  
3    False   False   False   False   False   False   False  


In [8]:
test.isAvailable(4, (2,2))

Not in index


False

In [9]:
test.isAvailable(3, (1,0))

True

# Things to try
min value of matrix
matrix first, break ties with hashmap

# Calculate minimize the loss E(P(x)/m*n) + E(M(x)/X)
- where x/m*n is the entropy for person,P, and P(x) is total available for person
- where E(M(x)/X) is the entropy of available spaces, M(x), against total spaces, X
# Operations:
How to keep track of positions
How to deal with failure end state
How to represent success state
Best way to find next position

# Failure States:
Easy: when X > m*n

Harder: when remaining states do not match remaining X states

# Success state:
When list is empty

In [10]:

""" 
schedule.insertMult(people[0], ((),(),(),(),(),(),(),(),(),()))
schedule.insertMult(people[1], ((),(),(),(),(),(),(),()))
schedule.insertMult(people[2], ((),(),(),(),(),()))
schedule.insertMult(people[3], ((),(),(),(),(),(),(),(),(),(),()))
schedule.insertMult(people[4], ((),(),(),(),(),(),()))
schedule.insertMult(people[5], ((),(),(),(),()))
schedule.insertMult(people[6], ((),(),()))
schedule.insertMult(people[7], (()))
schedule.insertMult(people[8], ((),(),(),(),(),()))
schedule.insertMult(people[9], ((),(),(),(),(),(),(),(),()))
schedule.insertMult(people[10], ((),(),(),()))
schedule.insertMult(people[11], ((),(),(),(),(),(),()))
schedule.insertMult(people[12], ((),(),(),(),(),()))
schedule.insertMult(people[13], ((),(),(),(),(),(),(),(),(),()))
schedule.insertMult(people[14], ((),(),(),(),(),(),(),()))
schedule.insertMult(people[15], ((),(),(),(),()))
schedule.insertMult(people[16], ((),(),(),(),(),(),(),()))
schedule.insertMult(people[17], ((),(),(),(),(),()))
schedule.insertMult(people[18], ((),(),(),(),(),(),(),(),()))
schedule.insertMult(people[19], ((),(),(),(),(),(),(),()))
schedule.insertMult(people[20], ((),(),(),(),(),(),(),(),(),(),(),(),()))
schedule.insertMult(people[21], ((),(),(),(),(),(),()))
schedule.insertMult(people[22], ((),(),(),(),(),(),()))
schedule.insertMult(people[23], ((),(),(),(),(),(),(),(),()))
schedule.insertMult(people[24], ((),(),(),(),(),(),(),()))
schedule.insertMult(people[25], ((),(),(),()))
"""
people = [Person() for x in range(26)]

In [11]:
GRID_SIZE = 48
schedule = Grid(GRID_SIZE)

def createTupList(maxval, size):
    s = set()
    while len(s) != size:
        s.add(tuple(np.random.randint(maxval, size = 2, dtype = np.uint64)))
    #print("\n", list(s))
    print("Length of list: ", len(s))
    return list(s)

for person in people:
    r = np.random.randint(GRID_SIZE**2, dtype = np.uint64)
    #print(person.id)
    schedule.insert(person.id, createTupList(48, r))
    
schedule.print()

Length of list:  1862
Length of list:  781
Length of list:  1853
Length of list:  1788
Length of list:  274
Length of list:  125
Length of list:  1447
Length of list:  1713
Length of list:  1633
Length of list:  1803
Length of list:  1834
Length of list:  2193
Length of list:  1340
Length of list:  346
Length of list:  1395
Length of list:  130
Length of list:  1033
Length of list:  146
Length of list:  2141
Length of list:  674
Length of list:  1824
Length of list:  1880
Length of list:  815
Length of list:  1216
Length of list:  937
Length of list:  466
    (0, 0)  (0, 1)  (0, 2)  (0, 3)  (0, 4)  (0, 5)  (0, 6)  (0, 7)  (0, 8)  \
0    False    True    True    True    True    True    True    True   False   
1    False   False   False    True   False    True   False   False    True   
2     True    True    True    True   False    True   False   False   False   
3     True    True    True    True    True    True    True    True   False   
4    False   False   False   False    True   Fal

In [12]:
for person in people:
    print(person.id, " Person available: ", schedule.getTotalbyidx(person.id))

0  Person available:  1862
1  Person available:  781
2  Person available:  1853
3  Person available:  1788
4  Person available:  274
5  Person available:  125
6  Person available:  1447
7  Person available:  1713
8  Person available:  1633
9  Person available:  1803
10  Person available:  1834
11  Person available:  2193
12  Person available:  1340
13  Person available:  346
14  Person available:  1395
15  Person available:  130
16  Person available:  1033
17  Person available:  146
18  Person available:  2141
19  Person available:  674
20  Person available:  1824
21  Person available:  1880
22  Person available:  815
23  Person available:  1216
24  Person available:  937
25  Person available:  466


In [13]:
for person in people:
    print(person.id, " Person available (by %): {:.4%}".format(schedule.getTotalbyidx(person.id)/schedule.getTotalSpots()))

0  Person available (by %): 80.8160%
1  Person available (by %): 33.8976%
2  Person available (by %): 80.4253%
3  Person available (by %): 77.6042%
4  Person available (by %): 11.8924%
5  Person available (by %): 5.4253%
6  Person available (by %): 62.8038%
7  Person available (by %): 74.3490%
8  Person available (by %): 70.8767%
9  Person available (by %): 78.2552%
10  Person available (by %): 79.6007%
11  Person available (by %): 95.1823%
12  Person available (by %): 58.1597%
13  Person available (by %): 15.0174%
14  Person available (by %): 60.5469%
15  Person available (by %): 5.6424%
16  Person available (by %): 44.8351%
17  Person available (by %): 6.3368%
18  Person available (by %): 92.9253%
19  Person available (by %): 29.2535%
20  Person available (by %): 79.1667%
21  Person available (by %): 81.5972%
22  Person available (by %): 35.3733%
23  Person available (by %): 52.7778%
24  Person available (by %): 40.6684%
25  Person available (by %): 20.2257%
